<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9D%D0%A1%5B2%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Нейронные сети [2]
####Вариант 5

In [14]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it

In [15]:
#функции активации
#гиперболический тангенс:
def hyperbolicTangentFunction(S, k = 0.9):
  return math.tanh(S / k)
#сигмоидная функция:
def sigmoidFunction(S, k = 0.9):
  return 1.0 / (1 + math.exp(- S * k))
#Пороговая функция:
def thresholdFunction(S, T = 0.7):
  return 1 if S >= T else 0

#=================================================================
def randomVector(size = 2):
  return [random.uniform(0, 1) for i in range(size)]

def randomNumber():
  return random.uniform(0, 1)

def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

#=================================================================
#функции для формирования таблицы данных
def x1(v):
  return math.sin(math.log(abs(v[0])) / v[2])

def x2(v):
  return math.cos(v[1]**3)

def x3(v):
  return math.sin(math.tan(v[0] / v[1]) - 3**(v[2]))

#def y1(x):
  #return (x[0] / x[1]) * math.sqrt(abs(x[0]))

#def y2(x):
  #return math.sqrt(abs(math.sin(x[2])))

def y3(x):
  return x[2] * x[0] / (x[1] + 0.1)

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -10, end = 10):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, elem = 100):
  vector = []
  for i in range(elem):
    vector.append(function(generationRandomVector()))
  return vector

#подставление x,v в выходные функции
def addVariants(function, x, elem = 100):
  vector = []
  for i in range(elem):
    vector.append(function(x[i]))
  return vector

In [24]:
#Коэффициент обучения [0;1]
learningRate = 0.3
#Количество эпох
epochs = 3
#количество слоёв
layer = 3
#количество входов/выходов соответственно
countX = 3
countY = 3
countRows = 10
#количество нейронов на каждом слое
neurons = [3, 2, countY]

networkCharact = [epochs, layer, neurons, learningRate]
inpOutCounts = [countX, countY]

#перечень функций:
inputFunctions = [x1, x2, x3]
outputFunctions = [y3]

#=================================================================
data = []
for i in range(len(inputFunctions)):
  data.append(createVariants(inputFunctions[i], countRows))
for i in range(len(outputFunctions)):
  data.append(addVariants(outputFunctions[i], np.array(data[0:3]).T, countRows))

data_df = pd.DataFrame(np.array(data).T, columns=['x1', 'x2', 'x3', 'y1'])

In [ ]:
#data_df

In [29]:
#charactNet = [кол-во эпох, кол-во слоёв, кол-во нейронов, коэф. обучения]
#sizeSystem = [кол-во входов, кол-во выходов]

def deltaRule(function, data, charactNet, sizeSystem):
  countEpochs = charactNet[0]
  countLayers = charactNet[1]
  countNeurons = charactNet[2]
  nabla = charactNet[3]

  indexOrder = [i for i in range(0, len(data))] #список всех индексов строк таблицы данных

  #si = [0] * sizeSystem[1]
  yi = [0] * sizeSystem[1]
  epsi = [0] * sizeSystem[1]
  level_i = 0
  epoch_i = 0
  k = 0

  #временная инициализация весов (только для однослойной сети!)
  #wi = [[randomNumber() for i in range(sizeSystem[0])] for j in range(countNeurons)] #синаптические веса
  wi = []

  while(epoch_i < countEpochs): #цикл по ЭПОХАМ!!!!!!!



    print("------------------------------------------------------------------------------------------------------------------")
    print("\tЭпоха №" + str(epoch_i + 1))
    random.shuffle(indexOrder)

    while(level_i < countLayers):
      neurons_i = countNeurons[level_i]

      if (epoch_i < 1):
        wi = [[randomNumber() for i in range(sizeSystem[0])] for j in range(neurons_i)] #синаптические веса



      while(k < len(data)): #цикл по строкам таблицы с данными

        print('================================================')
        print("Матрица весов для " + str(k + 1) + "-й итерации:")
        print(pd.DataFrame(wi))
        print("\nДля X = " + str(np.array(data.iloc[indexOrder[k], 0:sizeSystem[0]]).T) + '\n')

        eps = 0
        si = [0] * neurons_i

        for i in range(neurons_i): #по нейронам
          for j in range(sizeSystem[0]): #по входам

            #сделать проверку на соответствие
            si[i] += data[data.columns[j]][indexOrder[k]] * wi[i][j]

          #сделать проверку на сравнение количества нейронов и выходов для последнего слоя!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
          #yi[i] = function(si[i])
          #epsi[i] = data.iloc[indexOrder[k], sizeSystem[0] + i] - yi[i]
          #eps += (epsi[i]**2) / neurons_i

        #rez_i = pd.DataFrame([si, yi, data.iloc[k, sizeSystem[0]:], epsi], index=['S', 'Y расч.', 'Y теор.', 'EPS'],
                           #columns=['neuron ' + str(i+1) for i in range(sizeSystem[1])])
        print(si)
        #print(rez_i)
        #print("\nСКО: " + str(eps) + "\n\n")

        #вычисление весов для следующей итерации:
        for i in range(neurons_i):
          for j in range(sizeSystem[0]):
            wi[i][j] = wi[i][j] #+ nabla * epsi[i] * data[data.columns[j]][indexOrder[k]]

        k += 1

      level_i += 1

    epoch_i += 1
    k = 0

  #

In [30]:
#table = pd.DataFrame(it.product([0,1], repeat=countX), columns=['X' + str(i + 1) for i in range(countX)])
#table['Y1 = equivalence (~)'] = [equiv(table[table.columns[0]][i], table[table.columns[1]][i]) for i in range(2**neurons)]
#table['Y2 = conjunction (^)'] = [(table[table.columns[0]][i] and table[table.columns[1]][i]) for i in range(2**neurons)]

#deltaRule(hyperbolicTangentFunction, table, networkCharact, inpOutCounts)
#deltaRule(sigmoidFunction, table, layer, neurons, learningRate)
#deltaRule(thresholdFunction, table, layer, neurons, learningRate)

deltaRule(hyperbolicTangentFunction, data_df, networkCharact, inpOutCounts)

------------------------------------------------------------------------------------------------------------------
	Эпоха №1
Матрица весов для 1-й итерации:
          0         1         2
0  0.867877  0.371801  0.661532
1  0.605118  0.963288  0.045985
2  0.076580  0.433741  0.034318

Для X = [-0.3393814  -0.98974855 -0.97584316]

[-1.3080822251718098, -1.2036528863660851, -0.48877279298610665]
Матрица весов для 2-й итерации:
          0         1         2
0  0.867877  0.371801  0.661532
1  0.605118  0.963288  0.045985
2  0.076580  0.433741  0.034318

Для X = [ 0.17025467 -0.90360559  0.9712166 ]

[0.45428929700346465, -0.722746554116192, -0.34556258030550585]
Матрица весов для 3-й итерации:
          0         1         2
0  0.867877  0.371801  0.661532
1  0.605118  0.963288  0.045985
2  0.076580  0.433741  0.034318

Для X = [ 0.31858863 -0.6554602  -0.66462425]

[-0.40687493609713377, -0.469175900436817, -0.2827107179197106]
Матрица весов для 4-й итерации:
          0         1     

In [ ]:
#1-Количество входов и выходов могут быть различны +
#2-Реализовать расчёт промежуточных слоёв

In [ ]:
#Характеристики многослойной сети:
#1- вектор из кол-ва нейронов в каждом слое (в каждом слое м/б разное кол-во нейронов)
#2- кол-во выходов = кол-ву нейронов в крайнем слое
#3- входов может быть сколько угодно +
#4- в цикле: wij: i-номер нейрона, j-номер входа +
#5- размер s, eps равен количеству выходов +

In [ ]:
#1-Добавить условие для последнего слоя, где сравнивается кол-во нейронов и кол-во выходов
#2-